In [1]:
import timeit
import multiprocessing
import ray
import numpy as np
import unittest
import warnings 

from scipy.stats import pearsonr

from sklearn.datasets import make_classification, make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, explained_variance_score

import sklearn.model_selection
import pipecaster.metaprediction
from pipecaster import synthetic_data
from pipecaster.pipeline import Pipeline
from pipecaster.metaprediction import MetaClassifier, MetaRegressor
from pipecaster.model_selection import cross_val_score

In [37]:
import sklearn.model_selection 

seed = None

X, y = make_regression(n_samples=500, n_features=1000, n_informative=500, random_state=seed)
        
rgr = KNeighborsRegressor(n_neighbors=5, weights='uniform')
rgr.fit(X, y)
rgr_predictions = rgr.predict(X)

sklearn.model_selection.cross_val_score(rgr, X, y, cv=3, scoring='explained_variance')

array([-0.04430255, -0.16294141, -0.03633957])

In [13]:
mrgr = Pipeline(n_inputs)
layer0 = mrgr.get_next_layer()
layer0[:] = StandardScaler()
layer1 = mrgr.get_next_layer()
layer1[:] = KNeighborsRegressor(n_neighbors=5, weights='uniform')
layer2 = mrgr.get_next_layer()
layer2[:] = MetaRegressor('mean voting')

cross_val_score(mrgr, [X], y, prediction_method='predict', 
                 scoring_metric=explained_variance_score, cv=3, n_jobs=1, verbose=0, 
                 fit_params=None, error_score=np.nan)

[0.3343266057838239, 0.3112338234966967, 0.3576686030218642]

In [39]:
seed=None

n_inputs = 6
mean_accuracies, median_accuracies = [], []

sklearn_params = {'n_targets':1, 
          'n_samples':500, 
          'n_features':10, 
          'n_informative':5}

i = 6

Xs, y, _ = synthetic_data.make_multi_input_regression(n_informative_Xs=i, 
                        n_weak_Xs=0,
                        n_random_Xs=n_inputs - i,
                        weak_noise_sd=None,
                        seed = seed,
                        **sklearn_params                                   
                        )

mrgr = Pipeline(n_inputs)
layer0 = mrgr.get_next_layer()
layer0[:] = StandardScaler()
layer1 = mrgr.get_next_layer()
layer1[:] = KNeighborsRegressor(n_neighbors=5, weights='uniform')
layer2 = mrgr.get_next_layer()
layer2[:] = MetaRegressor('mean voting')

cross_val_score(mrgr, Xs, y, prediction_method='predict', 
                         scoring_metric=explained_variance_score, cv=3, n_jobs=1, verbose=0, 
                         fit_params=None, error_score=np.nan)

[0.16146119895625055, 0.15251922589133493, 0.18415469885178082]